In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [28]:
import os
import shutil

working_dir = "/kaggle/working"

# for deletion of wworking_dir
for filename in os.listdir(working_dir):
    file_path = os.path.join(working_dir, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f"Failed to delete {file_path}. Reason: {e}")

In [29]:
%%capture
%pip install -U -q --no-deps xformers trl transformers datasets peft accelerate huggingface_hub bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset, load_dataset
from peft import LoraConfig, PeftConfig, get_peft_model, LoraConfig, TaskType
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging,
                          Trainer,
                          DataCollatorForLanguageModeling)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [30]:
from huggingface_hub import login
login(token="your_hugging_face_token")

In [31]:
from kaggle_secrets import UserSecretsClient
secret_label = "HUGGING_FACE_WRITE_API_KEY"
secret_value = UserSecretsClient().get_secret(secret_label)

In [32]:
from huggingface_hub import snapshot_download

# Model details
model_id = "codellama/CodeLlama-7B-Instruct-hf"
local_dir = "/kaggle/working/codellama"

# Download full snapshot locally
model_path = snapshot_download(
    repo_id=model_id,
    local_dir=local_dir,
    local_dir_use_symlinks=False
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

LICENSE.txt:   0%|          | 0.00/7.71k [00:00<?, ?B/s]

original/consolidated.00.pth:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

...

original/tokenizer.model:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

USE_POLICY.md:   0%|          | 0.00/6.02k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

In [33]:
torch.cuda.empty_cache()

# running on kaggle
# Load tokenizer and model from local downloaded path
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    return_dict=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

Device set to use cuda:0


In [1]:
# # if not running on kaggle
# model_id = "codellama/CodeLlama-7b-Instruct-hf"

# tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     load_in_4bit=True,
#     device_map="auto",
#     torch_dtype="auto",
#     trust_remote_code=True
# )

In [ ]:
# Create inference pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [34]:
prompt = "How to convert COBOL code into Python?"
outputs = pipe(prompt, max_new_tokens=120, do_sample=True)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What is the tallest building in the world? What is the most expensive building in the world? The answer to these questions are two of the world's most expensive buildings, the Burj Khalifa in Dubai and the Burj Al Arab in Dubai. Both of these buildings are over 600 meters high, which makes them the tallest buildings in the world. The Burj Khalifa is also the tallest building in the world for a long time, until the Burj Dubai was built.
The Burj Khalifa is also the world's tallest building when measured from the top of the building to the top of the antenna. The Burj Al Arab


## Combine dataset 
| Dataset Name                    | HF ID                                            | Contains                  | Use                                |
| ------------------------------- | ------------------------------------------------ | ------------------------- | ---------------------------------- |
| **MainframeBench**              | `Fsoft-AIC/MainframeBench`                       | COBOL code + descriptions | Base COBOL understanding           |
| **The Stack**                   | `bigcode/the-stack`                              | COBOL + other languages   | Language variety + COBOL samples   |
| **Python Code Dataset**         | `jtatman/python-code-dataset-500k`               | Python code               | Target code corpus                 |
| **SantaCoder Fine-tuned COBOL** | `muhtasham/santacoder-finetuned-the-stack-cobol` | Pretrained model          | Base model for COBOL understanding |
| **General Code**                | `codeparrot/github-code`                         | Multi-language            | Extra fine-tuning                  |


In [37]:
# from datasets import load_dataset

# dataset = load_dataset("Fsoft-AIC/MainframeBench", split="train[:100]")  # sample subset

# # Dummy Python generation for demo (replace with real aligned translations)
# dataset = dataset.map(lambda x: {"tgt": "# Python translation of: " + x["code"]})


,statement,status
0,oh my gosh,Anxiety
1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,I've shifted my focus to something else but I'...,Anxiety
4,"I'm restless and restless, it's been a month n...",Anxiety


In [ ]:
from datasets import load_dataset, Dataset, concatenate_datasets

# Load relevant subsets
mainframe = load_dataset("Fsoft-AIC/MainframeBench", split="train")
stack = load_dataset("bigcode/the-stack", split="train[:1%]")  # 1% for feasibility
python_set = load_dataset("jtatman/python-code-dataset-500k", split="train")

# Filter COBOL and Python
stack_cobol = stack.filter(lambda x: x["lang"] == "cobol")
stack_python = stack.filter(lambda x: x["lang"] == "python")

# Combine relevant Python corpora
python_combined = concatenate_datasets([stack_python, python_set])

# Now build translation dataset


In [ ]:
# Dummy pairing for training (replace with aligned pairs if you have)
paired_data = []

for i in range(min(len(stack_cobol), len(python_combined))):
    paired_data.append({
        "input": stack_cobol[i]["content"],
        "output": python_combined[i]["content"]
    })

translation_dataset = Dataset.from_list(paired_data)


In [ ]:
# # we can upload custom dataset like following
# {"src": "COBOL code here", "tgt": "Equivalent Python code here"}
# {"src": "Another COBOL snippet", "tgt": "Translated Python code"}
# dataset = load_dataset("json", data_files={"train": "cobol_python_dataset.jsonl"})["train"]


In [ ]:
def format_prompt(example):
    return {
        "input_ids": tokenizer(
            f"""### Instruction:
Convert the following COBOL code to Python:

{example['src']}

### Response:
{example['tgt']}""",
            max_length=1024,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )["input_ids"][0]
    }

tokenized_dataset = dataset.map(format_prompt)


In [ ]:
# optional to save dataset locally
tokenized_dataset.to_json("codellama_cobol2python_dataset.json")

In [38]:
# # Shuffle the DataFrame and select only 3000 rows
# df = dataset.sample(frac=1, random_state=85).reset_index(drop=True).head(3000)

# # Split the DataFrame
# train_size = 0.8
# eval_size = 0.1

# # Calculate sizes
# train_end = int(train_size * len(df))
# eval_end = train_end + int(eval_size * len(df))

# # Split the data
# X_train = df[:train_end]
# X_eval = df[train_end:eval_end]
# X_test = df[eval_end:]

In [39]:
# Define the prompt generation functions for COBOL → Python conversion
def generate_prompt(data_point):
    return f"""
### Instruction:
Convert the following COBOL code to Python:

{data_point["src"]}

### Response:
{data_point["tgt"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
### Instruction:
Convert the following COBOL code to Python:

{data_point["src"]}

### Response:
""".strip()


In [40]:
# # Generate prompts for training and evaluation data
# X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
# X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)

# # Generate test prompts and extract true labels
# y_true = X_test.loc[:,'status']
# X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

/tmp/ipykernel_691/3727835688.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:,'text'] = X_train.apply(generate_prompt, axis=1)
/tmp/ipykernel_691/3727835688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_eval.loc[:,'text'] = X_eval.apply(generate_prompt, axis=1)


In [ ]:
# # Generate prompts for training and evaluation data
# X_train = X_train.assign(text=X_train.apply(generate_prompt, axis=1))
# X_eval = X_eval.assign(text=X_eval.apply(generate_prompt, axis=1))

# # Generate test prompts and extract true labels
# y_true = X_test['status'].copy()
# X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])

In [41]:
# X_train.status.value_counts()

status
Normal        1028
Depression     938
Anxiety        258
Bipolar        176
Name: count, dtype: int64

In [42]:
# # Convert to datasets
# train_data = Dataset.from_pandas(X_train[["text"]])
# eval_data = Dataset.from_pandas(X_eval[["text"]])

In [43]:
# train_data['text'][3]

'Classify the text into Normal, Depression, Anxiety, Bipolar, and return the answer as the corresponding mental health disorder label.\ntext: I am so sad. Everything in my work life is going fine, but my personal life is a wreck. No one ever takes me seriously because I am the funny friend. I do not want to talk to anyone anymore. I just want to die sometimes. Please help me. I have never had this feeling in my entire life. Why am I so sad\nlabel: Depression'

In [45]:
import bitsandbytes
print(bitsandbytes.__file__)

/usr/local/lib/python3.11/dist-packages/bitsandbytes/__init__.py


In [46]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Set env var to avoid fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

base_model_name = local_dir  # your snapshot_download path

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,  # helps reduce memory
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load model in 4-bit with automatic device placement
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map={"": 0},
    max_memory={0: "13GiB"}, # for kaggle space constraint
    quantization_config=bnb_config,
    trust_remote_code=True,
)

# Memory optimizations
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()  # saves memory during training

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

In [47]:
from transformers import pipeline
from tqdm import tqdm

def predict_code_translation(test_df, model, tokenizer):
    predictions = []
    
    for i in tqdm(range(len(test_df))):
        cobol_code = test_df.iloc[i]["src"]
        prompt = f"""
### Instruction:
Convert the following COBOL code to Python:

{cobol_code}

### Response:
""".strip()

        pipe = pipeline(
            task="text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            temperature=0.1,
            do_sample=False
        )

        result = pipe(prompt)
        generated_code = result[0]["generated_text"].split("### Response:")[-1].strip()
        predictions.append(generated_code)

    return predictions

# Example usage:
y_pred = predict_code_translation(X_test, model, tokenizer)


  1%|          | 3/300 [00:00<00:32,  9.00it/s]Device set to use cuda:0
Device set to use cuda:0
  2%|▏         | 5/300 [00:00<00:30,  9.58it/s]Device set to use cuda:0
Device set to use cuda:0
  3%|▎         | 8/300 [00:00<00:29,  9.84it/s]Device set to use cuda:0
Device set to use cuda:0
  4%|▎         | 11/300 [00:01<00:29,  9.83it/s]Device set to use cuda:0
Device set to use cuda:0
  5%|▍         | 14/300 [00:01<00:28,  9.95it/s]Device set to use cuda:0
Device set to use cuda:0
  6%|▌         | 18/300 [00:01<00:33,  8.45it/s]Device set to use cuda:0
Device set to use cuda:0
  7%|▋         | 21/300 [00:02<00:31,  9.00it/s]Device set to use cuda:0
Device set to use cuda:0
  8%|▊         | 23/300 [00:02<00:29,  9.49it/s]Device set to use cuda:0
Device set to use cuda:0
  8%|▊         | 25/300 [00:02<00:27,  9.87it/s]Device set to use cuda:0
Device set to use cuda:0
  9%|▉         | 27/300 [00:02<00:26, 10.12it/s]Device set to use cuda:0
Device set to use cuda:0
 10%|▉         | 29/300

In [48]:
from sklearn.metrics import accuracy_score, f1_score
from difflib import SequenceMatcher
import numpy as np

def evaluate_code_translation(y_true, y_pred):
    assert len(y_true) == len(y_pred), "Mismatch in number of samples"
    
    def code_similarity(a, b):
        return SequenceMatcher(None, a.strip(), b.strip()).ratio()
    
    similarities = [code_similarity(gt, pred) for gt, pred in zip(y_true, y_pred)]
    avg_similarity = np.mean(similarities)
    
    exact_matches = sum(1 for gt, pred in zip(y_true, y_pred) if gt.strip() == pred.strip())
    accuracy = exact_matches / len(y_true)

    print(f"Exact Match Accuracy: {accuracy:.3f}")
    print(f"Average Similarity Score: {avg_similarity:.3f}")
    
    # Optionally, show some examples
    for i in range(min(3, len(y_true))):
        print("\n--- Sample", i+1)
        print("True Output:\n", y_true[i])
        print("Predicted Output:\n", y_pred[i])
        print("Similarity Score:", code_similarity(y_true[i], y_pred[i]))

# Example usage:
evaluate_code_translation(y_true, y_pred)

Accuracy: 0.427
Accuracy for label Normal: 0.720
Accuracy for label Depression: 0.148
Accuracy for label Anxiety: 0.000
Accuracy for label Bipolar: 0.533

Classification Report:
              precision    recall  f1-score   support

      Normal       0.61      0.72      0.66       143
  Depression       0.40      0.15      0.22       115
     Anxiety       0.00      0.00      0.00        27
     Bipolar       0.11      0.53      0.19        15

   micro avg       0.45      0.43      0.44       300
   macro avg       0.28      0.35      0.26       300
weighted avg       0.45      0.43      0.41       300


Confusion Matrix:
[[103  21   0   6]
 [ 44  17   1  51]
 [ 17   4   0   6]
 [  6   1   0   8]]


In [49]:
import bitsandbytes as bnb

def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names:  # needed for 16 bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)
modules

['v_proj', 'gate_proj', 'k_proj', 'down_proj', 'up_proj', 'o_proj', 'q_proj']

In [66]:
# Avoid CUDA memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

output_dir = "CodeLlama-7B-Instruct-fine-tuned-model"

peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules  # You must define `modules`, e.g., ["q_proj", "v_proj"]
)

training_args = TrainingArguments(
    output_dir=output_dir,     # Path to save checkpoints
    per_device_train_batch_size=2,             # Small batch for large models
    gradient_accumulation_steps=4,             # Effective batch size = 8
    num_train_epochs=3,
    learning_rate=2e-4,
    weight_decay=0.001,
    
    bf16=True,                                 # If available, use bf16
    fp16=False,                                # Disable fp16 to avoid conflict with bf16

    gradient_checkpointing=True,               # Save memory
    optim="paged_adamw_32bit",                 # Efficient optimizer
    max_grad_norm=0.3,                         # Gradient clipping

    lr_scheduler_type="cosine",                # Cosine annealing
    warmup_ratio=0.03,                         # Warmup steps

    logging_steps=10,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=1,

    report_to="none",                          # Avoid wandb/logging integrations
    disable_tqdm=False                         # Show progress bars
)

sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=128,       # smaller = less memory
    # tokenizer=tokenizer,
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    # dataset_text_field="text",
    # tokenizer=tokenizer,
    # max_seq_length=128,
    # packing=False,
)

Converting train dataset to ChatML:   0%|          | 0/2400 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/2400 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2400 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2400 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/300 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [67]:
# To start the fine-tuning process:
trainer.train()

Step,Training Loss
1,1.484400
2,1.603100
3,1.946300
4,1.383600
5,1.334300
6,1.195500
7,1.336000
8,0.924700
9,1.078700
10,0.970500


TrainOutput(global_step=38, training_loss=1.291626883180518, metrics={'train_runtime': 752.1467, 'train_samples_per_second': 3.191, 'train_steps_per_second': 0.051, 'total_flos': 3200230769049600.0, 'train_loss': 1.291626883180518})

In [68]:
# Save trained model and tokenizer
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('llama-3.2-fine-tuned-model/tokenizer_config.json',
 'llama-3.2-fine-tuned-model/special_tokens_map.json',
 'llama-3.2-fine-tuned-model/tokenizer.json')

In [69]:
y_pred = predict(X_test, model, tokenizer)
evaluate(y_true, y_pred)

  0%|          | 0/300 [00:00<?, ?it/s]Device set to use cuda:0
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 300/300 [00:52<00:00,  5.76it/s]

Accuracy: 0.810
Accuracy for label Normal: 0.937
Accuracy for label Depression: 0.870
Accuracy for label Anxiety: 0.222
Accuracy for label Bipolar: 0.200

Classification Report:
              precision    recall  f1-score   support

      Normal       0.86      0.94      0.90       143
  Depression       0.76      0.87      0.81       115
     Anxiety       0.75      0.22      0.34        27
     Bipolar       1.00      0.20      0.33        15

   micro avg       0.81      0.81      0.81       300
   macro avg       0.84      0.56      0.60       300
weighted avg       0.82      0.81      0.79       300


Confusion Matrix:
[[134   8   0   0]
 [ 13 100   2   0]
 [  7  14   6   0]
 [  2  10   0   3]]


In [72]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

# Load your Hugging Face token from Kaggle secrets
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

# Login to Hugging Face Hub
login(token=hf_token)

In [73]:
base_model = base_model_name
fine_tuned_model = output_dir

In [74]:
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

In [75]:
# Merge adapter with base model
model = PeftModel.from_pretrained(base_model_reload, fine_tuned_model)
model = model.merge_and_unload()

In [76]:
cobol_code = """
       IDENTIFICATION DIVISION.
       PROGRAM-ID. HELLO.
       PROCEDURE DIVISION.
           DISPLAY 'HELLO, WORLD'.
           STOP RUN.
"""

prompt = f"""
### Instruction:
Convert the following COBOL code to Python:

{cobol_code}

### Response:
""".strip()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)

outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1)
generated_code = outputs[0]["generated_text"].split("### Response:")[-1].strip()

print(generated_code)

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Depression


In [86]:
# import shutil
# import os

# folder_path = "/kaggle/working/llama-3.2-fine-tuned-model"

# if os.path.exists(folder_path):
#     shutil.rmtree(folder_path)
#     print("Folder removed.")
# else:
#     print("Folder does not exist.")


In [87]:
model_dir = "CodeLlama-7B-Instruct-COBOL-to-Python"
model.save_pretrained(model_dir)
tokenizer.save_pretrained(model_dir)

('Llama-3.1-1B-Instruct-Mental-Health-Classification/tokenizer_config.json',
 'Llama-3.1-1B-Instruct-Mental-Health-Classification/special_tokens_map.json',
 'Llama-3.1-1B-Instruct-Mental-Health-Classification/tokenizer.json')

In [88]:
from huggingface_hub import CommitInfo

In [89]:
model.push_to_hub(model_dir, use_temp_dir=False)
tokenizer.push_to_hub(model_dir, use_temp_dir=False)

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dhirajpatra/Llama-3.1-1B-Instruct-Mental-Health-Classification/commit/d6402d8e3ac2ed34b90c951ed59d9eddc5c7135d', commit_message='Upload tokenizer', commit_description='', oid='d6402d8e3ac2ed34b90c951ed59d9eddc5c7135d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dhirajpatra/Llama-3.1-1B-Instruct-Mental-Health-Classification', endpoint='https://huggingface.co', repo_type='model', repo_id='dhirajpatra/Llama-3.1-1B-Instruct-Mental-Health-Classification'), pr_revision=None, pr_num=None)

In [90]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("dhirajpatra/" + model_dir)
tokenizer = AutoTokenizer.from_pretrained("dhirajpatra/" + model_dir)


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [94]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "dhirajpatra/" + model_dir

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [96]:
cobol_code = """
       IDENTIFICATION DIVISION.
       PROGRAM-ID. HELLO.
       PROCEDURE DIVISION.
           DISPLAY 'HELLO, WORLD'.
           STOP RUN.
"""

prompt = f"""### Instruction:
Convert the following COBOL code to Python:

{cobol_code}

### Response:
""".strip()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.1)
generated_code = outputs[0]["generated_text"].split("### Response:")[-1].strip()

print(generated_code)


Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Depression
